In [ ]:
# OBJETIVO DO NOTEBOOK
# 1º Normalizar dados númericos e categóricos
# 2º Caslibrar o número de grupos para o cluster (Otimização)
# 3º Treinar o modelo de cluster 
# 4º Descrever os grupos de modelo de cluster (Centróides)
# 5º Definir a qual grupo uma nova instância pertence

###########################################
# TRABALHO DE NORMALIZAÇÃO
# Data Since: 07/04/2023
# @Autor: Santian

!python.exe -m pip install --upgrade pip
!pip install pandas==1.5.2
!pip install matplotlib
!pip install scikit-learn

ABRE O ARQUIVO E REALIZA A LEITURA DO MESMO

In [ ]:
import pandas as pd
# LE O ARQUIVO
dataframe = pd.read_csv('codon_usage.csv\codon_usage.csv', sep=',')
# IMPRIME AS 5 PRIMEIRAS LINHAS
dataframe.head()